<a href="https://colab.research.google.com/github/abhishekv23/EE954-DL/blob/main/Assignment_1_Group1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Question 1

<p>Download the Fashion_MNIST dataset. You can find it on the official Fashion-MNIST website or by using PyTorch's torchvision.datasets module. Split the dataset into training, validation and testing sets. A common split is 80% of the data to train, 10% to validate, and 10% to test scenarios, but you can adjust this as needed. Normalize the images. This involves scaling the pixel values to a range between 0 and 1.</p>

---

In [ ]:
# !pip install torch
# !pip install torchvision

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os

import torch
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torch.utils.data import Dataset, random_split

In [ ]:
# Reading dataset
train = FashionMNIST('./data', train=True, download=True)

test_set = FashionMNIST('./data', train=False, download=True)

In [ ]:
# Spilt the Training and Validation dataset
train_size = int(0.8 * len(train))
val_size = int(0.2 * len(train))

train_set, val_set = random_split(train, [train_size, val_size])

In [ ]:
train_set

#### Using Pandas and Numpy

In [ ]:
# Using dataset in local
train = pd.read_csv('./Dataset/fashion-mnist_train.csv')

test = pd.read_csv('./Dataset/fashion-mnist_test.csv')

In [ ]:
train_cols = train.columns

In [ ]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Normalize training data
train_labels = train['label']

train_img = train.iloc[:, 1:]
train_img.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
train_images_flat = train_img.values.reshape(train_img.shape[0], -1)

In [ ]:
len(train_images_flat[1])

784

In [ ]:
# Normalize training data
train_images_flat = train_images_flat.astype('float32') / 255.0

In [ ]:
len(train_images_flat[1])

784

In [ ]:
normalized_df = pd.DataFrame.from_records(train_images_flat,
                                          columns = train_cols[1:])

normalized_df['labels'] = train_labels

In [ ]:
normalized_df.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,labels
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,9
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.019608,0.0,0.0,...,0.0,0.0,0.117647,0.168627,0.000000,0.0,0.0,0.0,0.0,6
3,0.0,0.0,0.0,0.003922,0.007843,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.003922,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,3


## Implement a MLP for classification. (total 40 marks)
a. Flatten the images into a single dimensional vector before feeding it to the model.
(1 marks)
b. Write a pre-processing module for all the images. (3 marks)
c. Write the Forward pass from scratch. Use of the inbuilt forward pass function will
result in 0 marks for this sub-question. (8 marks)
d. Write the Backward pass from scratch. Use the inbuilt back propagation function
will result in 0 marks for this sub-question (12 marks)
e. Write the module for cross entropy loss (1 marks)
f. Experiment with different hyperparameters like number of layers, dropout,
objective function, etc. and settle with a combination which performs the best for
the given problem. (15 Marks)

In [ ]:
#import torch
#import torchvision
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms

# Data Preprocessing (Part 2b)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# MLP Implementation
class MLP:
    def __init__(self, input_size, hidden_size, num_classes):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        # Initialize weights and biases
        self.W1 = torch.randn(input_size, hidden_size) * 0.01
        self.b1 = torch.zeros(hidden_size)
        self.W2 = torch.randn(hidden_size, num_classes) * 0.01
        self.b2 = torch.zeros(num_classes)

    # Forward Pass (Part 2c)
    def forward(self, x):
        x = x.view(-1, self.input_size)  # Flatten the image

        # First layer: input to hidden
        self.z1 = x @ self.W1 + self.b1
        self.a1 = self.relu(self.z1)  # Apply ReLU activation

        # Second layer: hidden to output
        self.z2 = self.a1 @ self.W2 + self.b2

        return self.z2

    def relu(self, z):
        return torch.max(torch.zeros_like(z), z)

    def relu_derivative(self, z):
        return (z > 0).float()

    # Backward Pass (Part 2d)
    def backward(self, x, y, outputs, learning_rate):
        m = y.size(0)

        dL_dz2 = outputs - y

        dL_dW2 = (self.a1.t() @ dL_dz2) / m
        dL_db2 = dL_dz2.mean(dim=0)

        dL_da1 = dL_dz2 @ self.W2.t()

        dL_dz1 = dL_da1 * self.relu_derivative(self.z1)

        dL_dW1 = (x.view(-1, self.input_size).t() @ dL_dz1) / m
        dL_db1 = dL_dz1.mean(dim=0)

        self.W1 -= learning_rate * dL_dW1
        self.b1 -= learning_rate * dL_db1
        self.W2 -= learning_rate * dL_dW2
        self.b2 -= learning_rate * dL_db2

# Cross Entropy Loss (Part 2e)
def cross_entropy_loss(outputs, labels):
    labels_one_hot = torch.zeros(labels.size(0), num_classes)
    labels_one_hot[range(labels.size(0)), labels] = 1
    loss = -torch.mean(torch.sum(labels_one_hot * torch.log_softmax(outputs, dim=1), dim=1))
    return loss

input_size = 28 * 28
hidden_size = 128
num_classes = 10
learning_rate = 0.01
num_epochs = 20

model = MLP(input_size, hidden_size, num_classes)

for epoch in range(num_epochs):
    for images, labels in train_loader:
        outputs = model.forward(images)

        loss = cross_entropy_loss(outputs, labels)

        # Backward pass
        model.backward(images, labels, outputs, learning_rate)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Validation and Testing
def evaluate(loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loader:
            outputs = model.forward(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
    return accuracy

val_accuracy = evaluate(val_loader)
test_accuracy = evaluate(test_loader)

print(f'Validation Accuracy: {val_accuracy:.2f}%')
print(f'Test Accuracy: {test_accuracy:.2f}%')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


NameError: name 'torchvision' is not defined